# Active Inference Design Agent (demo)

First, let us generate 10-20 audio signals from HA algorithm with different weights and within different contexts.

In [3]:
using Revise
using Rocket
using ReactiveMP
using GraphPPL
using Distributions
using LinearAlgebra
import ProgressMeter
using WAV
using Plots

In [214]:
include("helpers/aida_segmentation.jl")
include("helpers/aida_snr.jl")
include("helpers/aida_ar.jl")
include("models_inferences.jl")

batch_coupled_learning (generic function with 1 method)

In [65]:
fs = 8000

8000

## Let's obtain priors for the environment

In [241]:
babble, fs = wavread("sound/AIDA/training/sp01_babble_sn0.wav")
bbl_seg = get_frames(babble, fs)
bbl_totseg = size(bbl_seg, 1)

376

In [242]:
train, fs = wavread("sound/AIDA/training/sp11_train_sn0.wav")
tr_seg = get_frames(train, fs)
tr_totseg = size(tr_seg, 1)

391

In [52]:
# Priors for contexts were obtained through running VAD block
# prior for train noise
trmη = [0.4132104755921993 -0.16961911841949667]
trvη = [0.00972993440498344 -0.0027103005529199036; -0.0027103005529199036 0.004281987640515784]
trτ  = (41.0, 0.03644943410647206)
trτ  = (41.0, 0.05)

# prior for babble noise
bblmη = [1.1192255902602752 -0.43086292293101314]
bblvη = [0.007837790430663492 -0.005039080815241558; -0.005039080815241558 0.00596413119195013]
bblτ  = (41.0, 0.0029780512310493387);
bblτ  = (41.0, 0.05);

### VAD block

In [20]:
include("models/everything.jl")

lar_inference_ex (generic function with 1 method)

In [64]:
import WAV
include("helpers/aida_segmentation.jl")
include("helpers/aida_ar.jl")

ar_ssm (generic function with 1 method)

In [95]:
context_seg = bbl_seg
totseg = bbl_totseg
# context_seg = tr_seg
# totseg = tr_totseg

376

In [55]:
ar_order = 10
vmp_iter = 50
fe_ar = zeros(totseg, vmp_iter)
fe_gaussian = zeros(totseg, vmp_iter);

In [56]:
ProgressMeter.@showprogress for segnum in 1:totseg
    inputs, outputs = ar_ssm(context_seg[segnum, :], ar_order)
    γ, τ, θ, x, fe = lar_inference_ex(outputs, ar_order, vmp_iter)
    mθ, vθ = mean(θ), cov(θ)
    mγ = mean(γ)
    fe_ar[segnum, :] = fe
    
    x, γ, fe = inference_gaussian(outputs, vmp_iter, 1e4)
    mx, vx = mean(x), cov(x)
    mγ = mean(γ)
    fe_gaussian[segnum, :] = fe
end

Progress: 100%|█████████████████████████████████████████| Time: 0:08:47


In [57]:
# detect silent frames based on model comparison
vad = [x[end] < y[end] for (x, y) in zip(eachrow(fe_ar), eachrow(fe_gaussian))];

In [58]:
rsegs = findall(isequal(1), vad)
nsegs = findall(isequal(0), vad);

In [59]:
inv(var(sum([context_seg[nseg, :] for nseg in nsegs])))

17.938993996124832

In [60]:
cmx, cvx, cmη, cvη, cτ = lar_batch_learning(hcat([context_seg[nseg, :] for nseg in nsegs]...)', 2, 10, 1e-12);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In [64]:
println("$(mean(cmη, dims=1)), $(mean(cvη, dims=1)[1, :, :]), , $((cτ[end][1], cτ[end][2]))")

[1.1192255902602752 -0.43086292293101314], [0.007837790430663492 -0.005039080815241558; -0.005039080815241558 0.00596413119195013], , (41.0, 0.0029780512310493387)


In [8]:
# Priors for contexts were obtained through running VAD block
# prior for train noise
trmη = mean(cmη, dims=1)
trvη = mean(cvη, dims=1)[1, :, :]
trτ  = (cτ[end][1], cτ[end][2])

# prior for babble noise
bblmη = mean(cmη, dims=1)
bblvη = mean(cvη, dims=1)[1, :, :]
bblτ  = (cτ[end][1], cτ[end][2]);

LoadError: UndefVarError: cmη not defined

## Source seperation

In [296]:
# coupled AR model is deisgned to work with time-varying priors for both speech and environmental noise
# prior_to_priors map "static" priors to the corresponding matrices with equal elements
function prior_to_priors(mη, vη, τ, totseg)
    ar_order = size(mη, 2)
    rmη = zeros(totseg, ar_order)
    rvη = zeros(totseg, ar_order, ar_order)
    for segnum in 1:totseg
        rmη[segnum, :], rvη[segnum, :, :] = reshape(mη, (ar_order,)), vη
    end
    priors_eta = rmη, rvη
    priors_tau = [τ for _ in 1:totseg]
    priors_eta[1], priors_eta[2], priors_tau
end

prior_to_priors (generic function with 1 method)

In [243]:
trmη_arr, trvη_arr, trτ_arr = prior_to_priors(trmη, trvη, trτ, tr_totseg)
bblmη_arr, bblvη_arr, bblτ_arr = prior_to_priors(bblmη, bblvη, bblτ, bbl_totseg);

In [55]:
# This is dumb function that generates weights = [1.0, 0.0]
# agent will be here
function agent_weigths(sources_num)
    w = zeros(sources_num); w[1] = 1.0
    return w
end

agent_weigths (generic function with 1 method)

In [56]:
function HA_algorithm(segments, priors_η, priors_τ, ar_1_order, ar_2_order, vmp_its)
    """Source seperation based on coupled AR model. Inference is performed in batch manner

       segments: segmented audio signal
       priors_η:   matrix of means and covariances of AR coefficients (see output formal of prior_to_priors
       priors_τ:   array of tupes contatining the prior of environmental noise precision
       ar_1_order: order of speech signal
       ar_2_order: order of environmental noise signal
       vmp_its:    number of variational iterations
    """
    n_sources = 2
    totseg = size(segments, 1)
    l      = size(segments, 2) # dimensionality of the buffer
    
    rmx = zeros(totseg, l)
    rvx = zeros(totseg, l)
    rmθ = zeros(totseg, ar_1_order)
    rvθ = zeros(totseg, ar_1_order, ar_1_order)
    rγ = fill(tuple(.0, .0), totseg)
    
    rmz = zeros(totseg, l)
    rvz = zeros(totseg, l)
    rmη = zeros(totseg, ar_2_order)
    rvη = zeros(totseg, ar_2_order, ar_2_order)
    rτ = fill(tuple(.0, .0), totseg)
    
    fe  = zeros(totseg, vmp_its)
    
    rmo = zeros(totseg, l)
    
    # agent proposes weights according to its beliefs
    weights                           = agent_weigths(n_sources)
    ProgressMeter.@showprogress for segnum in 1:totseg
        prior_η                           = (priors_η[1][segnum, :], priors_η[2][segnum, :, :])
        prior_τ                           = priors_τ[segnum]
        γ, θ, zs, τ, η, xs, fe[segnum, :] = coupled_inference(segments[segnum, :], prior_η, prior_τ, ar_1_order, ar_2_order, vmp_its)
        mz, vz                            = mean.(zs), cov.(zs)
        mθ, vθ                            = mean(θ), cov(θ)
        rmz[segnum, :], rvz[segnum, :]    = first.(mz), first.(vz)
        rmθ[segnum, :], rvθ[segnum, :, :] = mθ, vθ
        rγ[segnum]                        = shape(γ), rate(γ)
        
        mx, vx                            = mean.(xs), cov.(xs)
        mη, vη                            = mean(η), cov(η)
        rmx[segnum, :], rvx[segnum, :]    = first.(mx), first.(vx)
        rmη[segnum, :], rvη[segnum, :, :] = mη, vη
        rτ[segnum]                        = shape(τ), rate(τ)
        
        # HA part
        speech = rmz[segnum, :] .* weights[1]
        noise  = rmx[segnum, :] .* weights[2]
        rmo[segnum, :] = speech .+ noise
    end
    rmz, rvz, rmθ, rvθ, rγ, rmx, rvx, rmη, rvη, rτ, fe, rmo
end

HA_algorithm (generic function with 1 method)

#### Obtain the outputs from HA

In [57]:
function get_sounds_fn(dir_name)
    file_names = []
    for (root, dirs, files) in walkdir(dir_name)
        for file in files
            push!(file_names, joinpath(root, file)) # path to files
        end
    end
    file_names
end

get_sounds_fn (generic function with 1 method)

In [58]:
training_files = get_sounds_fn("sound/AIDA/training/");

In [231]:
training_file = training_files[8]
println(training_file)
speech, fs = WAV.wavread(training_file)
speech_seg = get_frames(speech, fs)

sound/AIDA/training/sp08_babble_sn0.wav


380×80 Matrix{Float64}:
  0.0542314   0.0467544    0.00265511  …  -0.126011    -0.206244
  0.033723    0.0573443    0.0773339      -0.0227058   -0.0657979
  0.140538    0.106693     0.0360118       0.146092     0.0616169
  0.0662252  -0.018128    -0.032075       -0.0811182    0.0431227
  0.0480972   0.0445875    0.0498367      -0.0253609   -0.0185553
 -0.0292673  -0.0645466   -0.103793    …   0.14066      0.123875
 -0.0381481  -0.0107425    0.0104678       0.0936613    0.0746178
 -0.028077   -0.0843837   -0.0903348      -0.036256    -0.0307016
  0.0492569   0.0639973    0.0399182       0.050264    -0.0648824
  0.0670797   0.00317392   0.00521867      0.044496     0.0498978
 -0.101077   -0.0573138   -0.014069    …  -0.0697348   -0.0841395
  0.0554827   0.0452589   -0.0309763       0.156255     0.0937223
 -0.0675069   0.0357677    0.0458998      -0.156713    -0.0555437
  ⋮                                    ⋱               
  0.0116581  -0.00210578  -0.0115665      -0.0350963   -0.021149

In [312]:
import JLD
# r for recovered
for training_file in training_files
    println(training_file)
    speech, fs = WAV.wavread(training_file)
    speech_seg = get_frames(speech, fs)
    # choose priors
    priors_eta = occursin("babble", training_file) ? (bblmη_arr, bblvη_arr) : (trmη_arr, trvη_arr)
    priors_tau = occursin("babble", training_file) ? bblτ_arr : trτ_arr
    # make sure that 1D of priors and speech_seg are equal
    if size(priors_eta, 1) != size(speech_seg, 1)
        totseg = size(speech_seg, 1)
        priors_eta_m, priors_eta_v, priors_tau = prior_to_priors(priors_eta[1][1, :]', priors_eta[2][1, :, :], priors_tau[1], totseg)
        priors_eta = priors_eta_m, priors_eta_v
    end
    rmz, rvz, rmθ, rvθ, rγ, rmx, rvx, rmη, rvη, rτ, fe, rmo = HA_algorithm(speech_seg, priors_eta, priors_tau, 10, 2, 10);
    
    JLD.save("sound/AIDA/separated_jld/training/"*training_file[findfirst("sp", training_file)[1]:end][1:end-3]*"jld",
         "rmz", rmz, "rvz", rvz, "rmθ", rmθ, "rvθ", rvθ, "rγ", rγ, 
         "rmx", rmx, "rvx", rvx, "rmη", rmη, "rvη", rvη, "rτ", rτ,
         "fe", fe, "rmo", rmo, "filename", training_file,
         "audio", speech)
end

sound/AIDA/training/sp08_babble_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:39


sound/AIDA/training/sp09_babble_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:46


sound/AIDA/training/sp10_babble_sn0.wav


...

Progress: 100%|█████████████████████████████████████████| Time: 0:02:51


sound/AIDA/training/sp19_train_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:20


sound/AIDA/training/sp20_train_sn0.wav


Progress: 100%|█████████████████████████████████████████| Time: 0:03:14


### Preference learning stage

#### Generate outputs of HA from JLD files

In [313]:
training_jlds = get_sounds_fn("sound/AIDA/separated_jld/training/");

In [314]:
agent_weights = [[2.0, 1.0], [1.0, 0.0], [0.5, 0.5], [0.9, 0.3], [2.5, 1.0]]

5-element Vector{Vector{Float64}}:
 [2.0, 1.0]
 [1.0, 0.0]
 [0.5, 0.5]
 [0.9, 0.3]
 [2.5, 1.0]

In [315]:
for training_jld in training_jlds
    d = JLD.load(training_jld)
    rmz, rmx = d["rmz"], d["rmx"]
    filename = d["filename"]
    rz, rx = get_signal(rmz, fs), get_signal(rmx, fs)
    whgs = rand(agent_weights)
    ha_out = whgs[1] .* rz + whgs[2] .* rx
    
    WAV.wavwrite(ha_out, fs, "sound/AIDA/preference_learning/ha_out_$(whgs[1])_$(whgs[2])_"*filename[findfirst("sp", filename)[1]:end])
end

#### Create pairs weights<->appraisals

In [ ]:
# Recorded weights and appraisals 
weights = [[0.5, 0.5], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [2.0, 1.0], [2.0, 1.0], [2.5, 1.0], [2.5, 1.0], [2.5, 1.0]]
appraisals = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]

In [317]:
prl_files = get_sounds_fn("sound/AIDA/preference_learning/");

User gets to listen new audio samples with proposed weights. After each listening he/she evaluates the performance of HA output by binary feedback.

In [321]:
println(weights)
println(appraisals)

Any[[0.5, 0.5], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [0.9, 0.3], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [2.0, 1.0], [2.0, 1.0], [2.5, 1.0], [2.5, 1.0], [2.5, 1.0]]
Any[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0]


In [318]:
nnum = 8 # prefix for weights
appraisals = []
weights = []
for prl_file in prl_files
    
    WAV.wavplay(prl_file)
    println("How's HA output 1, 0 ?")
    appraisal = readline()
    push!(appraisals, parse(Float64, appraisal))
    
    # extract weights routine
    pref_id = findfirst("out_", prl_file)[end]
    weights_str = prl_file[pref_id+1:pref_id+nnum-1]
    push!(weights, parse.(Float64, split(weights_str, "_")))
end

How's HA output 1, 0 ?


stdin>  0.0


How's HA output 1, 0 ?


stdin>  0.0


How's HA output 1, 0 ?


...

stdin>  1.0


How's HA output 1, 0 ?


stdin>  0.0


How's HA output 1, 0 ?


stdin>  0.0


In [ ]:
test_files = get_sounds_fn("sound/AIDA/training/");

In [ ]:
# obtain HA output for test set (data for acting)
import JLD
# r for recovered
for test_file in test_files
    println(test_file)
    speech, fs = WAV.wavread(training_file)
    speech_seg = get_frames(speech, fs)
    # choose priors
    priors_eta = occursin("babble", training_file) ? (bblmη_arr, bblvη_arr) : (trmη_arr, trvη_arr)
    priors_tau = occursin("babble", training_file) ? bblτ_arr : trτ_arr
    rmz, rvz, rmθ, rvθ, rγ, rmx, rvx, rmη, rvη, rτ, fe, rmo = HA_algorithm(speech_seg, priors_eta, priors_tau, 10, 2, 10);
    
    JLD.save("sound/AIDA/separated_jld/test/"*test_file[findfirst("sp", test_file)[1]:end][1:end-3]*"jld",
         "rmz", rmz, "rvz", rvz, "rmθ", rmθ, "rvθ", rvθ, "rγ", rγ, 
         "rmx", rmx, "rvx", rvx, "rmη", rmη, "rvη", rvη, "rτ", rτ,
         "fe", fe, "rmo", rmo, "filename", test_file,
         "audio", speech)
end

In [ ]:
test_jlds = get_sounds_fn("sound/AIDA/separated_jld/test/");

In [ ]:
# Here agent plans and learns
for test_jld in test_jlds
    d = JLD.load(test_jld)
    rmz, rmx = d["rmz"], d["rmx"]
    filename = d["filename"]
    rz, rx = get_signal(rmz, fs), get_signal(rmx, fs)
    whgs = rand(agent_weights)
    ha_out = whgs[1] .* rz + whgs[2] .* rx
    # TODO: active loop
    WAV.wavwrite(ha_out, fs, "sound/AIDA/planning/ha_out_$(whgs[1])_$(whgs[2])_"*filename[findfirst("sp", filename)[1]:end])
end